<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="标题" style="width: 400px;"/> </a>

# 目标检测

### 使用部署

滑动窗口法需要部署经过训练的图像分类器，以在应用程序中将图像的 256X256 图块分类为“狗”或“猫”，而应用程序则会每次以 256X256 大小的窗口在图像上移动。虽然这一解决方案并不完美，但却可以：

1) 构建一个综合运用深度学习和传统编程的典型方法，从而打造出先前无法构建的应用程序
2) 引导启发学习神经网络架构

在本课程第一节中，您已学习如何成功*训练*及*部署*神经网络。您了解到，传统编程不足以支持图像分类，而深度学习不仅能够实现图像分类，还能以直观方式进行呈现。

到目前为止，您已采用现有神经网络和开源数据集（狗和猫），并创建出一个模型，用来对与初始数据集相像及不相像的新动物进行正确分类。这相当厉害！

在**此**任务中，您将学习如何通过以下方式以利用深度学习解决*图像分类之外*的问题：

- 将深度学习与传统计算机视觉相结合
- 修改神经网络的内部结构（实际数学运算的部分）
- 选择适合任务的神经网络

并进而解决第二项深度学习挑战：

**我们能否在图像中*检测*并*定位*物体？**

首先，我们会采用与上一节相同的方式，将模型实例化。我们引入模型和数据集工作目录来利用模型架构、经过训练的权重和预处理信息（如均值图像）。

In [ ]:
import time
import numpy as np #Data is often stored as "Numpy Arrays"
import matplotlib.pyplot as plt #matplotlib.pyplot allows us to visualize results
import caffe #caffe is our deep learning framework, we'll learn a lot more about this later in this task.
%matplotlib inline

MODEL_JOB_DIR = '/dli/data/digits/20180301-185638-e918'  ## Remember to set this to be the job directory for your model
DATASET_JOB_DIR = '/dli/data/digits/20180222-165843-ada0'  ## Remember to set this to be the job directory for your dataset

MODEL_FILE = MODEL_JOB_DIR + '/deploy.prototxt'                 # This file contains the description of the network architecture
PRETRAINED = MODEL_JOB_DIR + '/snapshot_iter_735.caffemodel'    # This file contains the *weights* that were "learned" during training
MEAN_IMAGE = DATASET_JOB_DIR + '/mean.jpg'                      # This file contains the mean image of the entire dataset. Used to preprocess the data.

# Tell Caffe to use the GPU so it can take advantage of parallel processing. 
# If you have a few hours, you're welcome to change gpu to cpu and see how much time it takes to deploy models in series. 
caffe.set_mode_gpu()
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

# load the mean image from the file
mean_image = caffe.io.load_image(MEAN_IMAGE)
print("Ready to predict.")

接着，我们将直接从前门监控摄像头中获取一张图像。请注意，此图像大于 256X256。我们想知道狗在此图像中位于*何处*。

In [ ]:
# Choose a random image to test against
#RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = '/dli/data/LouieReady.png'
input_image= caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

### 使用我们所学的函数：前向传播

这就是我们的关注所在。我们来看看这个函数：
<code>prediction = net.predict([grid_square])</code>。

与其他任一 [函数](https://www.khanacademy.org/computing/computer-programming/programming#functions)类似，<code>net.predict</code> 也会传递输入 <code>grid_square</code>，并返回输出 <code>预测</code>。但与其他函数不同，该函数并不遵循一组步骤，而是逐层开展矩阵数学运算，以将图像转换为概率向量。

运行以下单元，以从我们随机图像左上方的 256X256 <code>grid_square</code> 查看预测。

In [ ]:
X = 0
Y = 0

grid_square = input_image[X*256:(X+1)*256,Y*256:(Y+1)*256]
# subtract the mean image (because we subtracted it while we trained the network - more on this in next lab)
grid_square -= mean_image
# make prediction
prediction = net.predict([grid_square])
print prediction

这是所用网络最后一层的输出。层类型为“softmax”，其中每个单元的值越高，图像属于该类的可能性就越大。在上例中，如果向量的第二个单元高于第一个（且网络已经过训练），则该部分图像便可能包含*狗*。由于在本例中，第一个单元*远*高于第二个，因此很显然，网络并未在左上方的 256x256 网格中检测到狗。

可选：[探索 softmax 的数学运算。](https://en.wikipedia.org/wiki/Softmax_function)

接下来，我们将对此函数实施一些代码以对图像进行迭代，并对每个 grid_square 进行分类以创建热图。请注意，本节的课程关键在于，我们可以在此函数上构建“任何东西”，但凡所想，均可构建。

根据具体编程（尤其是 Python）操作的差异，您或许会从以下单元获得截然不同的信息。其核心是，这个块会把我们的输入图像分割成 256X256 大小的方块，并会在我们的狗分类器中对其逐一运行；之后此块会根据分类器所得结果创建新图像，其中没有狗时图像为蓝色，而有狗时则为红色。

In [ ]:
# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Calculate how many 256x256 grid squares are in the image
rows = input_image.shape[0]/256
cols = input_image.shape[1]/256

# Initialize an empty array for the detections
detections = np.zeros((rows,cols))

# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # subtract the mean image
        grid_square -= mean_image
        # make prediction
        prediction = net.predict([grid_square]) 
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections, interpolation=None)

# Display total time to perform inference
print 'Total inference time: ' + str(end-start) + ' seconds'

同样，这是随机选择的广域测试图像的输出结果和一个数组，显示输入到模型的每个非重叠 256x256 网格的预测类。

在我们更好地解决问题之前，请花一分钟时间回想刚才的操作，并回答以下问题：

**用您自己的话描述一下我们刚刚如何使用图像分类器检测物体？**

本节旨在说明任何深度学习解决方案都不过是从输入到输出的映射。这一实现手段使我们能够更易理解如何构建更复杂的解决方案。例如，Alexa、Google Assistant 和 Siri 之类的语音助手必须将原始声音数据转换为文本，并将文本转换为相应的理解，最后再将相应的理解转换为预期任务，例如播放您最喜爱的歌曲。

<a id='question1'></a>

其复杂性影响之广堪比任何已经或能够借助计算机科学构建的内容。请思考，在解决此类编码/问题所面临的挑战中，有哪些因素是我们未考虑到的，又该如何对其进行处理？

可能的答案：[单击此处](#answer1)

### 1.4 挑战性练习（可选）：

<a id='question-optional-exercise'></a>

1.使网格大小保持在 256x256，修改代码以增加网格间的重叠，并获得更精细的分类地图。

2.修改代码以将多个网格批量传递到网络中进行预测。

答案：[单击此处](#answer-optional-exercise)

正如当前所示，此滑动窗口法的优点在于我们可以仅使用基于局部的训练数据（可以更广泛地获取）训练检测器，且能帮助我们凭借当前的技能组合着手解决问题。

我们可以不断调整代码以确认自己能否继续通过蛮力解决问题（错误且缓慢），但也可学习更多有关深度学习的知识。我们来选择第二种方式。

## 方法 2 – 基于已有神经网络进行重建

请记住，深度神经网络的构建灵感源自于人脑。本节将说明如何*更改*网络以改变其行为和性能。就其核心而言，这些网络均由数学运算组成，改变数学运算即会改变大脑。本节不会全面深入地解释各类网络可完美解决各类问题的具体*原因*，因为这是一种会随时间逐渐累积的直觉知识。相反，本节将侧重介绍您在试验期间需牢记的限制条件。我们可能会介绍一些常见的层类型，但这并非本课程的内容要点。

我们先来探讨一下当前的网络 AlexNet 吧。

### 2.1 当前网络

这些网络的结构均会在某类*框架*中进行说明，而本例中使用的是 Caffe 框架。[框架种类数不胜数](https://developer.nvidia.com/deep-learning-frameworks)，其中每种框架都能使从业者对网络进行宏观描述，而不必对 GPU 进行物理编程以执行张量数学运算。

我们需要重新启用 DIGITS。

## [在此处启动 DIGITS](/digits)

如要在 DIGITS 中查看 AlexNet 的 Caffe 说明：

1) 选择模型“Dogs vs. Cats”（狗和猫）。
2) 选择“Clone Job”（克隆作业）（位于右上方并以蓝色字体表示）以将所有设置复制到新网络。克隆网络是实验中采用的一种重要迭代方法。
3) 当您在模型设置底部选择 AlexNet 后，请选定“Customize”（自定义）。

![](images/customize.png)

您将能看到某个*网络描述文件*，这是对 AlexNet 网络的 Caffe 说明。您可以随意查看该说明以弄清 AlexNet 网络的组织方式。在其他框架中，您也会看到类似内容，只是会有语法上的区别。

接下来，请在网络顶部，选择“Visualize”（可视化），以便可视化 AlexNet 网络。

![](images/visualize.png)

虽然此处内容可能对您而言不是十分重要，但有一点不容忽视：网络描述文件会直接告知哪些对象会显示在可视化内容中。对此，您可以亲自验证。将第一层的*名称*由*训练数据*改为*您的名称*，然后再次选择*可视化*。

如此一来您便可以掌控全局！

现在，我们再将名称改回*训练数据*以对重要内容（即数学运算）作出更改。

我们将替换 AlexNet 网络的部分层，以创建新功能并确保仍能训练网络。

### 2.2 网络手术

首先，我们来替换一个层，并确保各部分仍能正确连接。

在原型文本中，查找名称为“fc6”的层。您将可看到关于本层*功能*的大量介绍。该层内容将在后文有所提及，而我们当前只需将其替换掉。将单词“layer”（层）和右 [括号](#bracket "}") 之间以及下一个“layer”（层）之前的所有原型文本替换为以下内容：

```
layer {
  name: "conv6"
  type: "Convolution"
  bottom: "pool5"
  top: "conv6"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    pad: 0
    kernel_size: 6
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
```

现在来*可视化*网络，以将新网络和原始的 AlexNet 网络作一比较。以下列出原网络的开始部分供您参考：

![](images/AlexNet beginning.PNG)

以下是结束部分：

![](images/AlexNetEnd.PNG)

为什么这可能表明我们破坏了某些内容？

如要修复所造成的破坏，您认为我们可以采取什么措施？

### 2.3 完成工作

#### 规则 1：数据必须能够流动

我们删除了 fc6 层，但显然在可视化网络时其仍然存在。这是因为*其他层引用了 fc6*，其中这些层包括：

- fc7
- drop6
- relu6

查看 Caffe 网络描述文件，以了解这些层被*指示*连接到 fc6 的位置，并将其转而连接到新层 <code>conv6</code>。

**再次进行可视化。**您的网络应能再次显示，数据会像原始的 AlexNet 网络一般在相同的输入和输出之间流动。

当数据从输入流动到输出时，中间会经过许多执行运算（数学）的“隐藏层”。每一层的输出均需定义为下一层的输入。这会创建庞大的复合函数，并允许深度学习对输入和输出之间愈来愈复杂的真实关系进行建模。

一方面，我们可以使用所想的任何数学运算，只需使各部分保持连接，即可让数据从输入流动到输出。另一方面：

#### 规则 2：数学问题

我们可以抽些时间比较一下删除的层和添加的层。

我们删除了传统矩阵乘法的“全连接”层。虽然矩阵乘法知识所涉甚广，但我们所要了解的特性是矩阵乘法只适用于特定的矩阵大小。对于我们的问题而言，这意味着我们只能输入固定大小的图像（在本例中即为 256x256）。

我们添加了“卷积”层，这是输入矩阵上层的“过滤器”函数。虽然卷积知识同样所涉甚广，但我们要利用的特性是它们**不**只适用于特定的矩阵大小。对于我们的问题而言，这意味着如果我们将全连接层全部替换为卷积层，便可接受任何尺寸的图像。

**将 AlexNet 转换为“全卷积神经网络”后，我们将可把整个航拍图像输入至网络，而无需事先将其分割成网格。**

下面，我们来将 AlexNet 转换为全卷积神经网络（简称 FCN 容易令人产生误解）。使用以下 Caffe 文本将 `fc7` 至 `fc8` 替换为同等卷积层。

**重要须知：只能替换到“fc8”定义结束的地方。**

```
layer {
  name: "conv7"
  type: "Convolution"
  bottom: "conv6"
  top: "conv7"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu7"
  type: "ReLU"
  bottom: "conv7"
  top: "conv7"
}
layer {
  name: "drop7"
  type: "Dropout"
  bottom: "conv7"
  top: "conv7"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "conv8"
  type: "Convolution"
  bottom: "conv7"
  top: "conv8"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 2
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
```

为确保所有部分仍能保持正确连接，您是否必须对网络作出其他更改？若不确定，您可以再次对网络进行*可视化*以便于评估。若仍需要提示，请将鼠标指针悬停在 [此处](#rewiring "Remember to change the `bottom` blob of the `loss`, `accuracy` and `softmax` layers from 'fc8' to `conv8`.")。

在您认为所有部分均已正确连接后，即可开始训练新模型！

![](images/createmodel.PNG)

为您的模型命名并选择*create*（创建）；在训练模型时，请您观看下方视频以了解有关刚添加至网络的卷积的更多信息。

[![什么是卷积？](images/Convolutions.png)]（https://youtu.be/BBYnIoGrf8Y?t=13“什么是卷积”。）

若未出错，则说明您做得很棒！若有出错，请阅读错误描述。您或会从中了解到出错之处。通常情况下，错误会出现在 [此处](#rewiring "Remember to change the `bottom` blob of the `loss`, `accuracy` and `softmax` layers from 'fc8' to `conv8`.")。如果您只想要解决方案，请单击 [此处](#gu)。

<a id='fcn'></a>

**在您获得经过训练的模型后，请仿照第一节操作，将下方的 ##FIXME## 替换为新的“模型工作目录”。

In [ ]:
JOB_DIR = '##FIXME##'  ## Remember to set this to be the job directory for your model

您可以随意检查代码，以查看使用此模型和使用开箱即用的 AlexNet 之间存在何种差异，但要点在于我们要借助经过训练的模型输入*整张*图像而非多个网格。

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import copy
from scipy.misc import imresize
import time

MODEL_FILE = JOB_DIR + '/deploy.prototxt'                 # Do not change
PRETRAINED = JOB_DIR + '/snapshot_iter_735.caffemodel'    # Do not change                 

# Tell Caffe to use the GPU
caffe.set_mode_gpu()

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Initialize the Caffe model using the model trained in DIGITS
# This time the model input size is reshaped based on the randomly selected input image
net = caffe.Net(MODEL_FILE,PRETRAINED,caffe.TEST)
net.blobs['data'].reshape(1, 3, input_image.shape[0], input_image.shape[1])
net.reshape()
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

# This is just a colormap for displaying the results
my_cmap = copy.copy(plt.cm.get_cmap('jet')) # get a copy of the jet color map
my_cmap.set_bad(alpha=0) # set how the colormap handles 'bad' values

# Feed the whole input image into the model for classification
start = time.time()
out = net.forward(data=np.asarray([transformer.preprocess('data', input_image)]))
end = time.time()

# Create an overlay visualization of the classification result
im = transformer.deprocess('data', net.blobs['data'].data[0])
classifications = out['softmax'][0]
classifications = imresize(classifications.argmax(axis=0),input_image.shape,interp='bilinear').astype('float')
classifications[classifications==0] = np.nan
plt.imshow(im)
plt.imshow(classifications,alpha=.5,cmap=my_cmap)
plt.show()

# Display total time to perform inference
print 'Total inference time: ' + str(end-start) + ' seconds'

多次运行上述代码后，您会发现，全卷积神经网络在多数情况下均能比滑动窗口方法更精准地确定狗的位置。此外，FCN 的总推理用时约为 1.5 秒，而滑动窗口法则为 10 秒。我们能够在短时间内迅速取得更精准的结果！这在我们要将检测器部署至实时应用场景（例如登机安检）中时用处极大，因为我们的单一模型能够高效地一次完成检测和分类。

然而，这通常会遗漏本来属于狗的部分或错误检测出不是狗的片段。同样，我们还可以使用相应的策略来减少由背景杂波和狗引起的误报，例如适当的数据扩充和神经网络以外的其他数据工作等。

我们能够在网络内部更深入地解决该问题吗？

简短回答：能。

详细回答：通过使用*专用于*检测和定位狗的端到端物体检测网络。我们将使用通过 [COCO](http://cocodataset.org/#home) 数据集训练的 DetectNet。在此之前，请运行下列单元以释放内存。

In [3]:
try:
    del transformer
    del net
    del detections
except Exception as e:
    print e

name 'detections' is not defined


## 方法 3：DetectNet

在此实验开始时，我们采用的是 AlexNet，这是一款针对极具体*图像分类*问题的卓越解决方案。我们对其构建一些 Python 代码，然后开展轻微的网络手术，以此来完成一项在利用深度学习开始*目标检测*工作之前本不可能达成的任务。

不过，是否存在一款专门针对*目标检测*的卓越解决方案？我们能否建立一个模型，使其直接从输入（航拍照片）映射至预想的输出（定位和检测信息）？

此解决方案的存在将有助我们拓广对深度学习的定义，并能帮助您深入了解可借助其解决哪些*其他*问题。

本课程首先讨论的是深度学习得以实现的三个要素。

1) 深度神经网络
2) GPU
3) 大数据

当我们超出图像分类的讨论范围时，以上要素仍保持不变，但网络类型、数据和 GPU 的使用会有所不同。我们将从数据谈起，因为这是我们的工作流程在 DIGITS 中的组织方式。

### 数据差异

如要构建端到端监督式深度学习工作流程，我们需要已标记的数据。到目前为止，我们已将“已标记”定义为每个图像所属类别的指标。然而，标记数据的真正目的在于**创建输入输出映射。”

在本例中，我们希望输入“任意”尺寸的图像，且输出结果应能表明物体在图像中的所处位置。首先，请输入图像：

In [ ]:
input_image = caffe.io.load_image('/dli/data/train/images/488156.jpg') #!ls this directory to see what other images and labels you could test
plt.imshow(input_image)
plt.show()

然后，为其加上对应标签：

In [1]:
!cat '/dli/data/train/labels/488156.txt' #"cat" has nothing to do with the animals, this displays the text

dog 0 0 0 161.29 129.03 291.61 428.39 0 0 0 0 0 0 0
surfboard 0 0 0 31.25 36.44 410.41 640.0 0 0 0 0 0 0 0


注意：

1) 输入及其对应输出基于文件编号一一照应。
2) 所见向量由输入图像中狗的左上方和底部位置的 (x,y) 坐标构成。
3) 如果有足够的冲浪板数据，我们则可训练冲浪板检测器而非狗检测器！在本例中，我们会将数据集设置为只寻找狗。

将此数据集加载到 DIGITS 中。

从 [DIGITS](/digits) 主页面中，选择“Datasets”（数据集），然后添加新的**“目标检测”数据集。**请注意，这些参数看起来不同于进行分类时的参数。

打开“New Object Detection Dataset”（新的目标检测数据集）面板后，请使用以下图像的预处理选项（**请留意前部/尾部间距**）：

训练图像文件夹：/dli/data/train/images
训练标签文件夹：/dli/data/train/labels
验证图像文件夹：/dli/data/val/images
验证标签文件夹：/dli/data/val/labels/dog
补齐图像（宽 x 高）：640 x 640
自定义类：dontcare、dog
组名称：MS-COCO
数据集名称：coco-dog

![OD 数据摄取](images/OD_ingest.png)

注意：您不必等到数据集加载完成之后，才开始配置模型训练任务。

### 网络差异

当前人工“大脑”类型多种多样，专为*学习*各类不同内容而打造。[Deeplearning4J](https://deeplearning4j.org/neuralnetworktable) 具有一张可将网络类型映射到使用案例的巨型表格。方法 2 展示了改变限制条件将对网络行为带来何种影响。在方法 3 中，我们将展示研究、专业知识和迭代的优势，并会告知您无需成为其设计者，也可使用这些网络和模型。我们所用网络的设计者面临以下挑战：**从输出数据映射输入输出配对的最有效且最准确方法是什么？**

现在，我们将加载并可视化网络，以便了解这些网络的多样性。

选择屏幕左上方的 DIGITS 以返回 DIGITS 主屏幕。选择 Images（图像）-> Object Detection（目标检测）以创建新的目标检测**模型**。

选择您刚加载的数据集：“coco-dog”。

请先将训练次数改为 1、学习率改为 1e-100，具体原因稍后会加以说明。在您通常选择网络的位置，选择“Custom Network”（自定义网络），此操作应会打开一个空白窗口。

通过搜索“DetectNet deploy.prototxt”，我们找到了此网络相关说明。您的任务是将此链接 ([deploy.prototxt](../../../../edit/tasks/task5/task/deploy.prototxt)) 中的文本复制并粘贴到所显示的字段。请注意，当您执行这项任务时，将可轻易分享网络架构。

接下来，请选择“visualize”（可视化）以对网络进行可视化。同时，您需留意该网络与 AlexNet 在规模和复杂性上的区别。正如我们以上所构建的那样，DetectNet 实际上是一个全卷积神经网络 (FCN)，但配置后可精确产生此数据表示以作为输出。DetectNet 中的大部分层均与众所周知的 GoogLeNet 网络相同，


暂时先不要训练！

### 计算差异

随着网络深度以及任务复杂性的不断增加，相比我们目前为止所用的任何网络，这将需要*更多训练*。如果使用 NVIDIA Tesla K80，我们演示此工作流程性能所用的模型需要大约 16 小时进行训练，在 Volta 上用时更少，但在 CPU 上却永无尽头。

因此，我们不是从头训练，而是从预训练模型开始，并会以非常缓慢的学习率来进行每次训练（您已在上述步骤中作出此设置）。

如要加载已学习到的权重，请将权重加载到以下文件的字段（<code>预训练模型</code>）中：<pre>/dli/data/digits/snapshot_iter_38600.caffemodel</pre>

[](images/loadsnapshot.PNG)

命名您的模型并对其进行“训练”，以探究图像分类与物体检测之间的区别。

训练该模型所采用的性能衡量方法将不同于训练图像分类模型时报告的衡量法。平均精度均值 (mean Average Precision, mAP) 是一种组合衡量法，可用于衡量网络对狗的检测性能及其边框（定位）预测验证数据集的准确度。

您会发现，经过 100 次训练后，该模型不仅已收敛至较低的训练和验证损失值，而且可获得了较高的 mAP 得分。我们下面将使用测试图像来测试这一经过训练的模型，以验证它能否找到狗。

**将可视化方法设置为“边框”**，并将以下图像路径粘贴在：`/dli/data/BeagleImages/LouieTest1.jpg`。单击“Test One”（测试一）。您应能看到 DetectNet 已成功检测到 Louie，并绘制了边框。

若要尝试访问其他系列图像，请运行以下单元，并注意 Jupyter notebook 中的“！”会使这些单元起到命令行的作用。

In [ ]:
!ls /dli/data/BeagleImages

您可以随意测试 `/dli/data/BeagleImages` 文件夹中的其他图像。测试后，您会发现 DetectNet 能够使用精确绘制的边框准确检测到大多数狗，且误报率极低。同样值得注意的是，该模型还可用于检测多类物体！

请看以下一些要点：

1.正确的网络以及近便的工作数据远比自行创造解决方案效果更好。
2.正确的网络（有时甚至还包括预训练模型）也许可从 DIGITS 或框架的 [Model Zoo](https://github.com/BVLC/caffe/wiki/Model-Zoo) 中获取。

您可选用的一个问题解决框架如下所列：

- 确定他人是否已解决您的问题，若为此，则使用其模型
- 若为否，则确定他人是否已解决*类似*问题，若为此，则使用其网络和您本人数据
- 若为否，则确定您能否识别他人在解决您的问题时所存在的方案上的不足，并着力设计新网络
- 若为否，则使用现有解决方案和其他问题解决技术（如 python）来解决您的问题
- 无论方式为何，请继续试验并借助实验来提升您的技能组合！

在进入下一步之前，您大可庆祝一下现已取得的成就。

您目前了解到：

- 如何使用 DIGITS 训练多种类型的神经网络
- 您可以就网络训练*方式*作出何种更改以提高性能
- 如何综合运用深度学习与传统编程以解决新问题
- 如何修改网络的内部层次架构以更改功能和限制条件
- 查看他人是否已处理过类似问题，并确定您可以借鉴其解决方案的哪些部分

接下来，我们将深入探讨如何将深度神经网络*部署*到生产中以达成您计划解决的问题目标。此外，我们还将进一步了解您所需的信息、部署中的性能指标以及为最终用户维持（或提升）深度学习性能和速度的工具！

<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="标题" style="width: 400px;"/> </a>

## 问题答案：

<a id='answer1'></a>
### 答案 1

在使用具有非重叠网格的滑动窗口时，这还意味着我们的部分网格很有可能仅包含狗的一部分，而这可能导致错误分类。但当我们增加网格重叠时，滑动窗口法的计算时间也会迅速延长。为弥补计算时间延长这一不足，我们可以采取*批处理*输入策略，该策略能够利用 GPU 并行处理能力的内在优势。

[单击此处](#question1) 返回实验。

<a id='answer-optional-exercise'></a>

### 可选练习参考答案


In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import caffe
import time

MODEL_JOB_NUM = '##FIXME##'  ## Remember to set this to be the job number for your model
DATASET_JOB_NUM = '##FIXME##'  ## Remember to set this to be the job number for your dataset

MODEL_FILE = '/home/ubuntu/digits/digits/jobs/' + MODEL_JOB_NUM + '/deploy.prototxt'                 # Do not change
PRETRAINED = '/home/ubuntu/digits/digits/jobs/' + MODEL_JOB_NUM + '/snapshot_iter_735.caffemodel'    # Do not change
MEAN_IMAGE = '/home/ubuntu/digits/digits/jobs/' + DATASET_JOB_NUM + '/mean.jpg'                      # Do not change

# load the mean image
mean_image = caffe.io.load_image(MEAN_IMAGE)

# Choose a random image to test against
#RANDOM_IMAGE = str(np.random.randint(10))
IMAGE_FILE = '/dli/data/LouieReady.png' 

# Tell Caffe to use the GPU
caffe.set_mode_gpu()
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

# Load the input image into a numpy array and display it
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)
plt.show()

# Calculate how many 256x256 grid squares are in the image
rows = input_image.shape[0]/256
cols = input_image.shape[1]/256

# Subtract the mean image
for i in range(0,rows):
    for j in range(0,cols):
        input_image[i*256:(i+1)*256,j*256:(j+1)*256] -= mean_image
        
# Initialize an empty array for the detections
detections = np.zeros((rows,cols))
        
# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # make prediction
        prediction = net.predict([grid_square])
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print 'Total inference time (sliding window without overlap): ' + str(end-start) + ' seconds'

# define the amount of overlap between grid cells
OVERLAP = 0.25
grid_rows = int((rows-1)/(1-OVERLAP))+1
grid_cols = int((cols-1)/(1-OVERLAP))+1

print "Image has %d*%d blocks of 256 pixels" % (rows, cols)
print "With overlap=%f grid_size=%d*%d" % (OVERLAP, grid_rows, grid_cols)

# Initialize an empty array for the detections
detections = np.zeros((grid_rows,grid_cols))

# Iterate over each grid square using the model to make a class prediction
start = time.time()
for i in range(0,grid_rows):
    for j in range(0,grid_cols):
        start_col = int(j*256*(1-OVERLAP))
        start_row = int(i*256*(1-OVERLAP))
        grid_square = input_image[start_row:start_row+256, start_col:start_col+256]
        # make prediction
        prediction = net.predict([grid_square])
        detections[i,j] = prediction[0].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print ('Total inference time (sliding window with %f%% overlap: ' % (OVERLAP*100)) + str(end-start) + ' seconds'

# now with batched inference (one column at a time)
# we are not using a caffe.Classifier here so we need to do the pre-processing
# manually. The model was trained on random crops (256*256->227*227) so we
# need to do the cropping below. Similarly, we need to convert images
# from Numpy's Height*Width*Channel (HWC) format to Channel*Height*Width (CHW) 
# Lastly, we need to swap channels from RGB to BGR
net = caffe.Net(MODEL_FILE, PRETRAINED, caffe.TEST)
start = time.time()
net.blobs['data'].reshape(*[grid_cols, 3, 227, 227])

# Initialize an empty array for the detections
detections = np.zeros((rows,cols))

for i in range(0,rows):
    for j in range(0,cols):
        grid_square = input_image[i*256:(i+1)*256,j*256:(j+1)*256]
        # add to batch
        grid_square = grid_square[14:241,14:241] # 227*227 center crop        
        image = np.copy(grid_square.transpose(2,0,1)) # transpose from HWC to CHW
        image = image * 255 # rescale
        image = image[(2,1,0), :, :] # swap channels
        net.blobs['data'].data[j] = image
    # make prediction
    output = net.forward()[net.outputs[-1]]
    for j in range(0,cols):
        detections[i,j] = output[j].argmax()
end = time.time()
        
# Display the predicted class for each grid square
plt.imshow(detections)
plt.show()

# Display total time to perform inference
print 'Total inference time (batched inference): ' + str(end-start) + ' seconds'

完成后，请 [返回实验室](#question-optional-exercise)。

<a id='gu'></a>
# 全卷积神经网络解决方案

您可以将 AlexNet 中的**全部*文本替换为以下内容，以将其成功转换为全卷积神经网络。复制后，请单击 [此处](#fcn) 返回实验室。

```# AlexNet
name: "AlexNet"
layer {
  name: "train-data"
  type: "Data"
  top: "data"
  top: "label"
  transform_param {
    mirror: true
    crop_size: 227
  }
  data_param {
    batch_size: 128
  }
  include { stage: "train" }
}
layer {
  name: "val-data"
  type: "Data"
  top: "data"
  top: "label"
  transform_param {
    crop_size: 227
  }
  data_param {
    batch_size: 32
  }
  include { stage: "val" }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0
    }
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
layer {
  name: "norm1"
  type: "LRN"
  bottom: "conv1"
  top: "norm1"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "norm1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "pool1"
  top: "conv2"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 256
    pad: 2
    kernel_size: 5
    group: 2
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu2"
  type: "ReLU"
  bottom: "conv2"
  top: "conv2"
}
layer {
  name: "norm2"
  type: "LRN"
  bottom: "conv2"
  top: "norm2"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "pool2"
  type: "Pooling"
  bottom: "norm2"
  top: "pool2"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "conv3"
  type: "Convolution"
  bottom: "pool2"
  top: "conv3"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 384
    pad: 1
    kernel_size: 3
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0
    }
  }
}
layer {
  name: "relu3"
  type: "ReLU"
  bottom: "conv3"
  top: "conv3"
}
layer {
  name: "conv4"
  type: "Convolution"
  bottom: "conv3"
  top: "conv4"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 384
    pad: 1
    kernel_size: 3
    group: 2
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu4"
  type: "ReLU"
  bottom: "conv4"
  top: "conv4"
}
layer {
  name: "conv5"
  type: "Convolution"
  bottom: "conv4"
  top: "conv5"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 256
    pad: 1
    kernel_size: 3
    group: 2
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu5"
  type: "ReLU"
  bottom: "conv5"
  top: "conv5"
}
layer {
  name: "pool5"
  type: "Pooling"
  bottom: "conv5"
  top: "pool5"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "conv6"
  type: "Convolution"
  bottom: "pool5"
  top: "conv6"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    pad: 0
    kernel_size: 6
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu6"
  type: "ReLU"
  bottom: "conv6"
  top: "conv6"
}
layer {
  name: "drop6"
  type: "Dropout"
  bottom: "conv6"
  top: "conv6"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "conv7"
  type: "Convolution"
  bottom: "conv6"
  top: "conv7"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 4096
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "relu7"
  type: "ReLU"
  bottom: "conv7"
  top: "conv7"
}
layer {
  name: "drop7"
  type: "Dropout"
  bottom: "conv7"
  top: "conv7"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "conv8"
  type: "Convolution"
  bottom: "conv7"
  top: "conv8"
  param {
    lr_mult: 1.0
    decay_mult: 1.0
  }
  param {
    lr_mult: 2.0
    decay_mult: 0.0
  }
  convolution_param {
    num_output: 2
    kernel_size: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0.1
    }
  }
}
layer {
  name: "accuracy"
  type: "Accuracy"
  bottom: "conv8"
  bottom: "label"
  top: "accuracy"
  include { stage: "val" }
}
layer {
  name: "loss"
  type: "SoftmaxWithLoss"
  bottom: "conv8"
  bottom: "label"
  top: "loss"
  exclude { stage: "deploy" }
}
layer {
  name: "softmax"
  type: "Softmax"
  bottom: "conv8"
  top: "softmax"
  include { stage: "deploy" }
}
```

复制后，请单击 [此处](#fcn) 返回实验室。